In [ ]:
# https://github.com/philtabor/Multi-Agent-Deep-Deterministic-Policy-Gradients

!git clone https://github.com/openai/multiagent-particle-envs

Cloning into 'multiagent-particle-envs'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 242 (delta 0), reused 3 (delta 0), pack-reused 237
Receiving objects: 100% (242/242), 107.24 KiB | 11.92 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [ ]:
cd multiagent-particle-envs/

/content/multiagent-particle-envs


In [ ]:
!pip install gym==0.10.5
!pip install torch==1.4.0
!pip install numpy==1.14.5
!pip install -e .

!pip list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 34.7 MB/s 
  Created wheel for gym: filename=gym-0.10.5-py3-none-any.whl size=1581307 sha256=519b8eb9301a8a8a61c2ae9a6833afa1e4a6f88dae101f682e67b52bc1e9f613
  Stored in directory: /root/.cache/pip/wheels/7a/2c/df/a05b548a40fae16ca400ecbeda0067e1a296499c1fbd7e0c9a
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.1
    Uninstalling gym-0.25.1:
      Successfully uninstalled gym-0.25.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 7.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does n

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/multiagent-particle-envs
  Running setup.py develop for multiagent
Package                       Version                      Location
----------------------------- ---------------------------- ---------------------------------
absl-py                       1.2.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1

In [ ]:
import os
import numpy as np
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from make_env import make_env


Let's inspect the environment: 
- There are three agents
- The observation space is Box(8,) for the adversary red agent
- The observation space is Box(10,) for the two collaborative green agents
- The possible actions are 5: don't move, move up,down,left,right

In [ ]:
env = make_env("simple_adversary")
print("number of agents:", env.n)
print("observation space:", env.observation_space)
print("action space:", env.action_space)
print("n actions", env.action_space[0].n)

number of agents: 3
observation space: [Box(8,), Box(10,), Box(10,)]
action space: [Discrete(5), Discrete(5), Discrete(5)]
n actions 5


Actually, the obervation space is basically a list of three arrays of 8, 10 and 10 double each

In [ ]:
observation = env.reset()
print(observation)

[array([-1.14583482, -0.56097481, -1.69303519,  0.95920345, -0.57124765,
        0.48163571, -0.57639631, -0.30269139]), array([-1.12178754,  0.47756774, -0.57458717, -1.04261051, -1.12178754,
        0.47756774,  0.57124765, -0.48163571, -0.00514866, -0.7843271 ]), array([-1.11663889,  1.26189484, -0.56943851, -0.25828342, -1.11663889,
        1.26189484,  0.57639631,  0.30269139,  0.00514866,  0.7843271 ])]


Let's first define the MultiAgentReplayBuffer class

The parameters are related to 
- the memory size of the replay buffer (max_size)
- the number of agents (n_agents)
- the dimension of ... (actor_dims) ???
- the batch_size
- the number possible of actions

Other members that are initialize are:
- state_memory:     a matrix of mem_size x critic_dims
- new_state_memory: a matrix of mem_size x critic_dims
- reward_memory:    a matrix of mem_size x n_agents
- terminal_memory:  a matrix of mem_size x n_agents

So:

- state_memory and new_state_memory pertain to the critic and are centralized,
- reward_memory and terminal_memory pertain to the actors and are decentralized

Upon initialization of the class (\__init\__), we also initialize the actor memory (init_actor_memory).

Here we initialize the actor_state_memory, actor_new_state_memory and actor_action_memory.

These are basically lists of n_agents matrices of sizes mem_size x actor_dims for each actor

When we store a transition, we need:
- the observations of each agent of the current state (raw\_obs) that is an array of length n_agents.
- the observations of each agent of the new state (raw\_obs\_) that is an array of length n_agents.
- the actions of each agent (action) that is an array of length n_agents.

For each agent agent_idx, we store:
- actor_state_memory[agent_idx][index] = raw_obs[agent_idx]
- actor_new_state_memory[agent_idx][index] = raw_obs_[agent_idx]
- actor_action_memory[agent_idx][index] = action[agent_idx]

where the $index \in [0,mem\_size]$ identifies the label of the trasition we are storing in the replay buffer

Moreover, we have the parameters:
- state: an array of length critic_dims
- state_: an array of length critic_dims
- reward: an array of length n_agents
- done: an array of length n_agents (bool)

that we store in 

- state_memory[index][0:critic_dims] = state[0:critic_dims]
- new_state_memory[index][0:critic_dims] = state_[0:critic_dims]
- reward_memory[index][0:critic_dims] = reward[0:critic_dims]
- terminal_memory[index][0:critic_dims] = done[0:critic_dims]

Finally, we update the couter, getting ready to store the new transition


When we want to sample from the ReplayBuffer, we first check
- max_mem = min(self.mem_cntr, self.mem_size) 

that is 

- max_mem = mem_cntr, if we have not yet recorded enough trasition to full the mem_size
- otherwise, max_mem = mem_size, that is we sample from the whole possible memory

From max_mem, we sample a list batch of batch_size indices that we use to sample

- states = self.state_memory[batch]
- rewards = self.reward_memory[batch]
- states_ = self.new_state_memory[batch]
- terminal = self.terminal_memory[batch]

For each agent, we define the lists actor_states, actor_new_states, actions that contain n_agents arrays, each corresponding to the batch sampled from actor_state_memory, actor_new_state_memory, actor_action_memory of each agent

In [ ]:
class MultiAgentReplayBuffer:
    def __init__(self, max_size, critic_dims, actor_dims, n_actions, n_agents, batch_size):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.n_agents = n_agents
        self.actor_dims = actor_dims
        self.batch_size = batch_size
        self.n_actions = n_actions

        self.state_memory = np.zeros((self.mem_size, critic_dims))
        self.new_state_memory = np.zeros((self.mem_size, critic_dims))
        self.reward_memory = np.zeros((self.mem_size, n_agents))
        self.terminal_memory = np.zeros((self.mem_size, n_agents), dtype=bool)

        self.init_actor_memory()

    def init_actor_memory(self):
        self.actor_state_memory = []
        self.actor_new_state_memory = []
        self.actor_action_memory = []

        for i in range(self.n_agents):
            self.actor_state_memory.append(
                            np.zeros((self.mem_size, self.actor_dims[i])))
            self.actor_new_state_memory.append(
                            np.zeros((self.mem_size, self.actor_dims[i])))
            self.actor_action_memory.append(
                            np.zeros((self.mem_size, self.n_actions)))


    def store_transition(self, raw_obs, state, action, reward, 
                               raw_obs_, state_, done):
        # this introduces a bug: if we fill up the memory capacity and then
        # zero out our actor memory, the critic will still have memories to access
        # while the actor will have nothing but zeros to sample. Obviously
        # not what we intend.
        # In reality, there's no problem with just using the same index
        # for both the actor and critic states. I'm not sure why I thought
        # this was necessary in the first place. Sorry for the confusion!

        #if self.mem_cntr % self.mem_size == 0 and self.mem_cntr > 0:
        #    self.init_actor_memory()
        
        index = self.mem_cntr % self.mem_size

        for agent_idx in range(self.n_agents):
            self.actor_state_memory[agent_idx][index] = raw_obs[agent_idx]
            self.actor_new_state_memory[agent_idx][index] = raw_obs_[agent_idx]
            self.actor_action_memory[agent_idx][index] = action[agent_idx]

        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        self.mem_cntr += 1

    def sample_buffer(self):
        max_mem = min(self.mem_cntr, self.mem_size)

        batch = np.random.choice(max_mem, self.batch_size, replace=False)

        states = self.state_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]

        actor_states = []
        actor_new_states = []
        actions = []
        for agent_idx in range(self.n_agents):
            actor_states.append(self.actor_state_memory[agent_idx][batch])
            actor_new_states.append(self.actor_new_state_memory[agent_idx][batch])
            actions.append(self.actor_action_memory[agent_idx][batch])

        return actor_states, states, actions, rewards, \
               actor_new_states, states_, terminal

    def ready(self):
        if self.mem_cntr >= self.batch_size:
            return True






In [ ]:
class CriticNetwork(nn.Module):
    def __init__(self, beta, input_dims, fc1_dims, fc2_dims, 
                    n_agents, n_actions, name, chkpt_dir):
        super(CriticNetwork, self).__init__()

        self.chkpt_file = os.path.join(chkpt_dir, name)

        self.fc1 = nn.Linear(input_dims+n_agents*n_actions, fc1_dims)
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        self.q = nn.Linear(fc2_dims, 1)

        self.optimizer = optim.Adam(self.parameters(), lr=beta)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
 
        self.to(self.device)

    def forward(self, state, action):
        x = F.relu(self.fc1(T.cat([state, action], dim=1)))
        x = F.relu(self.fc2(x))
        q = self.q(x)

        return q

    def save_checkpoint(self):
        T.save(self.state_dict(), self.chkpt_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.chkpt_file))


class ActorNetwork(nn.Module):
    def __init__(self, alpha, input_dims, fc1_dims, fc2_dims, 
                 n_actions, name, chkpt_dir):
        super(ActorNetwork, self).__init__()

        self.chkpt_file = os.path.join(chkpt_dir, name)

        self.fc1 = nn.Linear(input_dims, fc1_dims)
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        self.pi = nn.Linear(fc2_dims, n_actions)

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
 
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        pi = T.softmax(self.pi(x), dim=1)

        return pi

    def save_checkpoint(self):
        T.save(self.state_dict(), self.chkpt_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.chkpt_file))

Each agent has 4 Neural Network corresponding to:
- actor network
- target actor network
- critic network
- target critic network 

To choose an action we use the actor network to which we add noise.

To update the parameters of the neural networks, we make a copy of the target network parameters and have them slowly track those of the learned networks via “soft updates”, polyak averaging with parater $(1-\tau) = \rho$

After having constructed the actor_state_dict, we update the target actor network using
- self.target_actor.load_state_dict(actor_state_dict)

After having constructed the critic_state_dict, we update the target actor network using
- self.target_critic.load_state_dict(critic_state_dict)

In [ ]:
class Agent:
    def __init__(self, actor_dims, critic_dims, n_actions, n_agents, agent_idx, chkpt_dir,
                    alpha=0.01, beta=0.01, fc1=64, 
                    fc2=64, gamma=0.95, tau=0.01):
        self.gamma = gamma
        self.tau = tau
        self.n_actions = n_actions
        self.agent_name = 'agent_%s' % agent_idx
        self.actor = ActorNetwork(alpha, actor_dims, fc1, fc2, n_actions, 
                                  chkpt_dir=chkpt_dir,  name=self.agent_name+'_actor')
        self.critic = CriticNetwork(beta, critic_dims, 
                            fc1, fc2, n_agents, n_actions, 
                            chkpt_dir=chkpt_dir, name=self.agent_name+'_critic')
        self.target_actor = ActorNetwork(alpha, actor_dims, fc1, fc2, n_actions,
                                        chkpt_dir=chkpt_dir, 
                                        name=self.agent_name+'_target_actor')
        self.target_critic = CriticNetwork(beta, critic_dims, 
                                            fc1, fc2, n_agents, n_actions,
                                            chkpt_dir=chkpt_dir,
                                            name=self.agent_name+'_target_critic')

        self.update_network_parameters(tau=1)

    def choose_action(self, observation):
        state = T.tensor([observation], dtype=T.float).to(self.actor.device)
        actions = self.actor.forward(state)
        noise = T.rand(self.n_actions).to(self.actor.device)
        action = actions + noise

        return action.detach().cpu().numpy()[0]

    def update_network_parameters(self, tau=None):
        if tau is None:
            tau = self.tau

        target_actor_params = self.target_actor.named_parameters()
        actor_params = self.actor.named_parameters()

        target_actor_state_dict = dict(target_actor_params)
        actor_state_dict = dict(actor_params)
        for name in actor_state_dict:
            actor_state_dict[name] = tau*actor_state_dict[name].clone() + \
                    (1-tau)*target_actor_state_dict[name].clone()

        self.target_actor.load_state_dict(actor_state_dict)

        target_critic_params = self.target_critic.named_parameters()
        critic_params = self.critic.named_parameters()

        target_critic_state_dict = dict(target_critic_params)
        critic_state_dict = dict(critic_params)
        for name in critic_state_dict:
            critic_state_dict[name] = tau*critic_state_dict[name].clone() + \
                    (1-tau)*target_critic_state_dict[name].clone()

        self.target_critic.load_state_dict(critic_state_dict)

    def save_models(self):
        self.actor.save_checkpoint()
        self.target_actor.save_checkpoint()
        self.critic.save_checkpoint()
        self.target_critic.save_checkpoint()

    def load_models(self):
        self.actor.load_checkpoint()
        self.target_actor.load_checkpoint()
        self.critic.load_checkpoint()
        self.target_critic.load_checkpoint()

In [ ]:
class MADDPG:
    def __init__(self, actor_dims, critic_dims, n_agents, n_actions, 
                 scenario='simple',  alpha=0.01, beta=0.01, fc1=64, 
                 fc2=64, gamma=0.99, tau=0.01, chkpt_dir='tmp/maddpg/'):
        self.agents = []
        self.n_agents = n_agents
        self.n_actions = n_actions
        chkpt_dir += scenario 
        for agent_idx in range(self.n_agents):
            self.agents.append(Agent(actor_dims[agent_idx], critic_dims,  
                            n_actions, n_agents, agent_idx, alpha=alpha, beta=beta,
                            chkpt_dir=chkpt_dir))


    def save_checkpoint(self):
        print('... saving checkpoint ...')
        for agent in self.agents:
            agent.save_models()

    def load_checkpoint(self):
        print('... loading checkpoint ...')
        for agent in self.agents:
            agent.load_models()

    def choose_action(self, raw_obs):
        actions = []
        for agent_idx, agent in enumerate(self.agents):
            action = agent.choose_action(raw_obs[agent_idx])
            actions.append(action)
        return actions

    def learn(self, memory):
        if not memory.ready():
            return

        # for agent i=1 to N do
           # Sample a random minibatch of S samples from D
        actor_states, states, actions, rewards, \
        actor_new_states, states_, dones = memory.sample_buffer()

        device = self.agents[0].actor.device
        
        states = T.tensor(states, dtype=T.float).to(device)
        actions = T.tensor(actions, dtype=T.float).to(device)
        rewards = T.tensor(rewards).to(device)
        states_ = T.tensor(states_, dtype=T.float).to(device)
        dones = T.tensor(dones).to(device)

        all_agents_new_actions = []
        all_agents_new_mu_actions = []
        old_agents_actions = []

        for agent_idx, agent in enumerate(self.agents):
            new_states = T.tensor(actor_new_states[agent_idx], 
                                 dtype=T.float).to(device)

            new_pi = agent.target_actor.forward(new_states)

            all_agents_new_actions.append(new_pi)
            mu_states = T.tensor(actor_states[agent_idx], 
                                 dtype=T.float).to(device)
            pi = agent.actor.forward(mu_states)
            all_agents_new_mu_actions.append(pi)
            old_agents_actions.append(actions[agent_idx])

        new_actions = T.cat([acts for acts in all_agents_new_actions], dim=1)
        mu = T.cat([acts for acts in all_agents_new_mu_actions], dim=1)
        old_actions = T.cat([acts for acts in old_agents_actions],dim=1)

        for agent_idx, agent in enumerate(self.agents):
            # Get the critic value and the target critic value
            critic_value_ = agent.target_critic.forward(states_, new_actions).flatten()
            critic_value_[dones[:,0]] = 0.0
            critic_value = agent.critic.forward(states, old_actions).flatten()

            # To be able to define y_i
            target = rewards[:,agent_idx] + agent.gamma*critic_value_
            # and the critic loss
            critic_loss = F.mse_loss(target, critic_value)
            # Update critic by minimizing the loss
            agent.critic.optimizer.zero_grad()
            critic_loss.backward(retain_graph=True)
            agent.critic.optimizer.step()

            # Update actor using the sampled policy gradient:
            # we can just perform gradient ascent (with respect to policy parameters only)
            actor_loss = agent.critic.forward(states, mu).flatten()
            actor_loss = -T.mean(actor_loss)
            agent.actor.optimizer.zero_grad()
            actor_loss.backward(retain_graph=True)
            agent.actor.optimizer.step()

            # Update the parameters
            agent.update_network_parameters()

In [ ]:
def obs_list_to_state_vector(observation):
    state = np.array([])
    for obs in observation:
        state = np.concatenate([state, obs])
    return state

In [ ]:
#scenario = 'simple'
os.makedirs("tmp/maddpg/simple_adversary/", exist_ok=True)
scenario = 'simple_adversary'
env = make_env(scenario)
n_agents = env.n
actor_dims = []
for i in range(n_agents):
    actor_dims.append(env.observation_space[i].shape[0])
critic_dims = sum(actor_dims)

# action space is a list of arrays, assume each agent has same action space
n_actions = env.action_space[0].n
maddpg_agents = MADDPG(actor_dims, critic_dims, n_agents, n_actions, 
                           fc1=64, fc2=64,  
                           alpha=0.01, beta=0.01, scenario=scenario,
                           chkpt_dir='tmp/maddpg/')#

memory = MultiAgentReplayBuffer(1000000, critic_dims, actor_dims, n_actions, n_agents, batch_size=1024)

PRINT_INTERVAL = 500
N_GAMES = 50000
MAX_STEPS = 25
total_steps = 0
score_history = []
evaluate = False
best_score = 0

if evaluate:
    maddpg_agents.load_checkpoint()

for i in range(N_GAMES):
    obs = env.reset()
    score = 0
    done = [False]*n_agents
    episode_step = 0
    while not any(done):
        if evaluate:
            env.render()
            #time.sleep(0.1) # to slow down the action for the video
        actions = maddpg_agents.choose_action(obs)
        obs_, reward, done, info = env.step(actions)

        state = obs_list_to_state_vector(obs)
        state_ = obs_list_to_state_vector(obs_)

        if episode_step >= MAX_STEPS:
            done = [True]*n_agents

        memory.store_transition(obs, state, actions, reward, obs_, state_, done)

        if total_steps % 100 == 0 and not evaluate:
            maddpg_agents.learn(memory)

        obs = obs_

        score += sum(reward)
        total_steps += 1
        episode_step += 1

    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    if not evaluate:
        if avg_score > best_score:
            maddpg_agents.save_checkpoint()
            best_score = avg_score
    if i % PRINT_INTERVAL == 0 and i > 0:
        print('episode', i, 'average score {:.1f}'.format(avg_score))

episode 500 average score -22.2
episode 1000 average score -25.6
episode 1500 average score -24.8
episode 2000 average score -22.5
episode 2500 average score -23.9
episode 3000 average score -26.9
episode 3500 average score -22.4
episode 4000 average score -22.9
episode 4500 average score -23.3
episode 5000 average score -23.5
episode 5500 average score -22.5
episode 6000 average score -23.1
episode 6500 average score -23.5
episode 7000 average score -21.0
episode 7500 average score -19.5
episode 8000 average score -19.9
episode 8500 average score -22.5
episode 9000 average score -23.6
episode 9500 average score -21.4
episode 10000 average score -22.7
episode 10500 average score -21.5
episode 11000 average score -23.9
episode 11500 average score -21.4
episode 12000 average score -23.1
episode 12500 average score -22.8
episode 13000 average score -24.0
episode 13500 average score -25.8
episode 14000 average score -23.4
episode 14500 average score -21.8
episode 15000 average score -19.8


KeyboardInterrupt: ignored